In [1]:
#!pip install catboost
#!pip install xgboost
#!pip install openml
#!pip install numba

In [2]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load dataset_sales dataset
dataset = openml.datasets.get_dataset(42183, download_data=True, download_qualities=True, download_features_meta_data=True)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
categorical_feature_indices = [idx for idx, idx_bool in enumerate(categorical_indicator) if idx_bool]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))

X_train.head()

Training set size: 6872
Validation set size: 1718
Test set size: 2148


,productId,machineId,temp,weather_condition_id,isholiday,daysoff,year,month,day,week_day,avail0,avail1,avail2,stdv
9369,0.548387,0.0,0.559935,0.994048,0,0.0,1,0.727273,0.500000,0.000000,0.50,1.00,0.67,1
6363,0.806452,0.5,0.655238,0.992063,0,0.0,1,0.545455,0.033333,0.166667,1.00,0.83,1.00,1
9859,0.064516,1.0,0.470842,0.992063,0,0.0,1,0.727273,0.933333,1.000000,1.00,1.00,1.00,0
2307,0.548387,1.0,0.148488,0.992063,0,0.0,1,0.181818,0.666667,0.500000,0.83,0.50,0.83,2
2139,0.967742,1.0,0.337473,0.396825,0,0.0,1,0.181818,0.433333,0.500000,0.50,1.00,1.00,1


In [4]:
from GRANDE import GRANDE

params = {
        'depth': 5, # tree depth
        'n_estimators': 2048, # number of estimators / trees

        'learning_rate_weights': 0.005, # learning rate for leaf weights
        'learning_rate_index': 0.01, # learning rate for split indices
        'learning_rate_values': 0.01, # learning rate for split values
        'learning_rate_leaf': 0.01, # learning rate for leafs (logits)

        'optimizer': 'adam', # optimizer
        'cosine_decay_steps': 0, # decay steps for lr schedule (CosineDecayRestarts)

        'loss': 'mse', # loss function (default 'crossentropy' for binary & multi-class classification and 'mse' for regression)
        'focal_loss': False, # use focal loss {True, False}
        'temperature': 0.0, # temperature for stochastic re-weighted GD (0.0, 1.0)

        'from_logits': True, # use logits for weighting {True, False}
        'use_class_weights': True, # use class weights for training {True, False}

        'dropout': 0.0, # dropout rate (here, dropout randomly disables individual estimators of the ensemble during training)

        'selected_variables': 0.8, # feature subset percentage (0.0, 1.0)
        'data_subset_fraction': 1.0, # data subset percentage (0.0, 1.0)
}

args = {
    'epochs': 1_000, # number of epochs for training
    'early_stopping_epochs': 25, # patience for early stopping (best weights are restored)
    'batch_size': 64,  # batch size for training

    'cat_idx': categorical_feature_indices, # put list of categorical indices
    'objective': 'regression', # objective / task {'binary', 'classification', 'regression'}
    
    'random_seed': 42,
    'verbose': 1,       
}

model_grande = GRANDE(params=params, args=args)

model_grande.fit(X_train=X_train,
          y_train=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict(X_test)

2024-03-26 10:21:15.016663: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 10:21:19.911137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-26 10:22:00.787376: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-26 10:22:00.787852: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9546 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5


Epoch 1/1000


I0000 00:00:1711444936.282159 3119067 service.cc:145] XLA service 0x7f691c003790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1711444936.282354 3119067 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-03-26 10:22:16.688349: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-26 10:22:18.358735: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


 18/108 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.8053

I0000 00:00:1711444940.390766 3119067 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


108/108 ━━━━━━━━━━━━━━━━━━━━ 23s 86ms/step - loss: 0.8023 - val_loss: 0.7273
Epoch 2/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.5520 - val_loss: 0.5437
Epoch 3/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.4041 - val_loss: 0.4775
Epoch 4/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.3338 - val_loss: 0.4243
Epoch 5/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.2760 - val_loss: 0.3835
Epoch 6/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.2504 - val_loss: 0.3496
Epoch 7/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.2260 - val_loss: 0.3140
Epoch 8/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.2095 - val_loss: 0.3045
Epoch 9/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.2025 - val_loss: 0.2849
Epoch 10/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.1971 - val_loss: 0.2837
Epoch 11/1000
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.1884 - val_loss: 0.2591
Epoch 12/1000
108/108 ━━━━━━━━━━

In [5]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    return sample_weights

In [6]:
try:
    y_train = y_train.values.codes.astype(np.float64)
    y_valid = y_valid.values.codes.astype(np.float64)
    y_test = y_test.values.codes.astype(np.float64)
except:
    y_train = y_train.values.astype(np.float64)
    y_valid = y_valid.values.astype(np.float64)
    y_test = y_test.values.astype(np.float64)

In [7]:
binary_indices = []
low_cardinality_indices = []
high_cardinality_indices = []
num_columns = []
for column_index, column in enumerate(X_train.columns):
    if column_index in categorical_feature_indices:
        if len(X_train.iloc[:,column_index].unique()) <= 2:
            binary_indices.append(column)
        elif len(X_train.iloc[:,column_index].unique()) < 5:
            low_cardinality_indices.append(column)
        else:
            high_cardinality_indices.append(column)
    else:
        num_columns.append(column)     
cat_columns = [col for col in X_train.columns if col not in num_columns]

In [8]:
if len(num_columns) > 0:
    mean_train_num = X_train[num_columns].mean(axis=0).iloc[0]
    X_train[num_columns] = X_train[num_columns].fillna(mean_train_num)
    X_valid[num_columns] = X_valid[num_columns].fillna(mean_train_num)
    X_test[num_columns] = X_test[num_columns].fillna(mean_train_num)
if len(cat_columns) > 0:
    mode_train_cat = X_train[cat_columns].mode(axis=0).iloc[0]
    X_train[cat_columns] = X_train[cat_columns].fillna(mode_train_cat)
    X_valid[cat_columns] = X_valid[cat_columns].fillna(mode_train_cat)
    X_test[cat_columns] = X_test[cat_columns].fillna(mode_train_cat)

X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

In [9]:
encoder_ordinal = ce.OrdinalEncoder(cols=binary_indices)
encoder_ordinal.fit(X_train)
X_train = encoder_ordinal.transform(X_train)
X_valid = encoder_ordinal.transform(X_valid)  
X_test = encoder_ordinal.transform(X_test)       

encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
from xgboost import XGBRegressor
model_xgb = XGBRegressor(n_estimators=1000, early_stopping_rounds=20)
model_xgb.fit(X_train, 
                y_train, 
                sample_weight=calculate_sample_weights(y_train), 
                eval_set=[(X_valid, y_valid)], 
                sample_weight_eval_set=[calculate_sample_weights(y_valid)]
             )
if args['objective'] == 'regression':
    preds_xgb = model_xgb.predict(X_test)
else:
    preds_xgb = model_xgb.predict_proba(X_test)

[0]	validation_0-rmse:21.32764
[1]	validation_0-rmse:17.83969
[2]	validation_0-rmse:14.79749
[3]	validation_0-rmse:13.09891
[4]	validation_0-rmse:11.70010
[5]	validation_0-rmse:11.22561
[6]	validation_0-rmse:10.70027
[7]	validation_0-rmse:10.36325
[8]	validation_0-rmse:10.00313
[9]	validation_0-rmse:9.87188
[10]	validation_0-rmse:9.81904
[11]	validation_0-rmse:9.75513
[12]	validation_0-rmse:9.68747
[13]	validation_0-rmse:9.63047
[14]	validation_0-rmse:9.62859
[15]	validation_0-rmse:9.61208
[16]	validation_0-rmse:9.58495
[17]	validation_0-rmse:9.63782
[18]	validation_0-rmse:9.61342
[19]	validation_0-rmse:9.72494
[20]	validation_0-rmse:9.68926
[21]	validation_0-rmse:9.66877
[22]	validation_0-rmse:9.65851
[23]	validation_0-rmse:9.63861
[24]	validation_0-rmse:9.63711
[25]	validation_0-rmse:9.62849
[26]	validation_0-rmse:9.61850
[27]	validation_0-rmse:9.61490
[28]	validation_0-rmse:9.62154
[29]	validation_0-rmse:9.61162
[30]	validation_0-rmse:9.60180
[31]	validation_0-rmse:9.60386
[32]	vali

In [11]:
from catboost import CatBoostRegressor, Pool

model_catboost = CatBoostRegressor(n_estimators=1000, 
                                    early_stopping_rounds=20)
train_data = Pool(
        data=X_train_raw,
        label=y_train,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_train)
    )

eval_data = Pool(
        data=X_valid_raw,
        label=y_valid,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_valid),
    )

model_catboost.fit(X=train_data, 
                   eval_set=eval_data)

if args['objective'] == 'regression':
    preds_catboost = model_catboost.predict(X_test_raw)
else:
    preds_catboost = model_catboost.predict_proba(X_test_raw)


Learning rate set to 0.068916
0:	learn: 13.8938300	test: 14.8375117	best: 14.8375117 (0)	total: 71.1ms	remaining: 1m 11s
1:	learn: 13.4868750	test: 14.6356384	best: 14.6356384 (1)	total: 81.8ms	remaining: 40.8s
2:	learn: 13.1038555	test: 14.3930520	best: 14.3930520 (2)	total: 88.2ms	remaining: 29.3s
3:	learn: 12.7778368	test: 14.2964681	best: 14.2964681 (3)	total: 92.8ms	remaining: 23.1s
4:	learn: 12.4106060	test: 14.1245314	best: 14.1245314 (4)	total: 99.4ms	remaining: 19.8s
5:	learn: 12.1370113	test: 13.9665072	best: 13.9665072 (5)	total: 105ms	remaining: 17.4s
6:	learn: 11.8227111	test: 13.8241105	best: 13.8241105 (6)	total: 109ms	remaining: 15.5s
7:	learn: 11.5206139	test: 13.7326919	best: 13.7326919 (7)	total: 115ms	remaining: 14.3s
8:	learn: 11.1639396	test: 13.5592696	best: 13.5592696 (8)	total: 122ms	remaining: 13.4s
9:	learn: 10.9145692	test: 13.4335573	best: 13.4335573 (9)	total: 127ms	remaining: 12.6s
10:	learn: 10.5669160	test: 13.2766710	best: 13.2766710 (10)	total: 132ms	

In [12]:
if args['objective'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif args['objective'] == 'classification':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_grande))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_grande))

    print('MAE GRANDE:', mean_absolute_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_xgb))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_xgb))

    print('MAE XGB:', mean_absolute_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_catboost))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_catboost))

    print('MAE CatBoost:', mean_absolute_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

MAE GRANDE: 1.7583798882681565
R2 Score GRANDE: 0.8311929764673647


MAE XGB: 2.3254189944134076
R2 Score XGB: 0.7261986930299364


MAE CatBoost: 4.0139664804469275
R2 Score CatBoost: 0.3879855183782981




In [13]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()